# Introduction
This project is to use a statistical model to predict the wind stress anomalies by sea surface temperature anomalies (SSTA). The relationship can be represented as $\boldsymbol{\tau_s} = \boldsymbol{C} \boldsymbol{T}$, where $\boldsymbol{\tau_s}$ and $\boldsymbol{T}$ are state vectors of wind stress anomolies and SSTA, respectively, and $\boldsymbol{C}$ is a constant coefficient (a matrix). Our goal is to derive $\boldsymbol{C}$ using Singular Value Decomposition (SVD) analysis on a training dataset, and apply the relationship on a testing dataset.

This project is going to:

1. Formulate a statistical atmosphere model for predicting surface wind stress anomalies for given SST anomalies;
2. Validate the simulated surface wind stress anomalies against observations;
3. Perform sensitivity tests to see how the results depend on the number of SVDs used to compute the anomalies;
4. Discuss whether the results make physical sense and why;

# Dataset and Method
Surface temperature ($T_s$), zonal wind stress ($u$), meridional wind stress ($v$), and a land mask dataset are given. $T_s, u, v$ are three-dimensional, with two space dimensions and one time dimension. Datasets are divided into two parts: data in 1948-1999 are used as training dataset, and data in 2000-2017 are used as testing dataset.

We follow the following steps to perform the SVD analysis:

1. Normalize all the anomaly fields by dividing each of variables by its own standard deviation;
2. Form a state vector for SSTA ($\boldsymbol{T}$), and another one for zonal and meridional wind stress anomaly ($\boldsymbol{\tau_s}$). Note that $\boldsymbol{\tau_s}$ contains both wind stress component and has larger dimension than $\boldsymbol{T}$;
3. Form a normalized covariance matrix, $\boldsymbol{A=\tau _s T'}$, whose dimension should be $M\times N$, where $M$ is the length for $\boldsymbol{\tau_s}$ and $N$ is length for $\boldsymbol{T}$;
4. Apply SVD on $\boldsymbol{A}$ by calling Matlab’s SVD function, which results in $\boldsymbol{A=USV'}$, $S$ is a diagonal matrix with diagonal elements representing singular values - explained squared covariance in each SVD, $\boldsymbol{U}$ contains all wind stress singular vectors and $V$ contains all SST singular vectors, which are self-orthogonal, i.e., $\boldsymbol{VV'=I}$ and $\boldsymbol{UU'=I}$. 

# Results
## SVD analysis
Although the SVD MATLAB scripts are provided, I rewrite them in Python in order to better understand the SVD method. The Python script is below:

In [ ]:
import 